<a href="https://colab.research.google.com/github/aminKMT/LLM-Wage-Predictor/blob/main/week7_Dataminig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining for Wage Dataset

## Import the excel data:

In [ ]:
import pandas as pd
df = pd.read_excel("/W2_CPSwage_Pr.xlsx", sheet_name="CPS wages")
df.head()

## Checking Categorical Data

In [ ]:
for cols in df.columns:
  if cols in ('south', 'Gender','Union','Race', "Occup", "Sector", "Marr" ):
     print(df[cols].value_counts())


## Categorical data cleaning:
### Elimination

In [13]:
# eliminating wring Gende Type
df = df[df.Gender != 'Ale']
# eliminating wring Race:
df = df[(df.Race != 4) & (df.Race != 5)]

## Descriptive statistics for non-categorical variables

In [ ]:
df[['Educ',	'Experience',	'Wage', 'Age']].describe()

In [ ]:
#check if any varible has missing varibel(if yes, how many !?):
df.isna().sum()

In [ ]:
#If you only want to replace missing values in one numeric column
df['Wage'].fillna(df['Wage'].mean(), inplace=True)
#Fill missing values in all numeric columns
#df.fillna(df.mean(numeric_only=True), inplace=True)

# Check the dataframe:
df.isna().sum()

## Handeling years of experience outliers:

In [ ]:

# creating the histogram before removing outl
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.hist(df['Experience'], bins=20, edgecolor='black')
plt.title('Distribution of Years of Experience')
plt.xlabel('Years of Experience')
plt.ylabel('Frequency')
plt.show()

In [56]:
#Interquabtile for outlier of years of experience
Q1 = df['Experience'].quantile(0.25)
Q3 = df['Experience'].quantile(0.75)
IQR = Q3 - Q1

# Define acceptable range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df_no_outliers = df[(df['Experience'] >= lower_bound) & (df['Experience'] <= upper_bound)]

In [ ]:
# creating the histogram AFTER removing outliers
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.hist(df_no_outliers['Experience'], bins=20, edgecolor='black')
plt.title('Distribution of Years of Experience')
plt.xlabel('Years of Experience')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# convert Gender values to number
# Making a hash map(Dectionary)
map = {
    'Female' :1,
    'Male' : 0
}

df_no_outliers['Gender'] = df_no_outliers['Gender'].replace(map)
# lets see the column
df_no_outliers['Gender']


In [ ]:
df_no_outliers['Gender'].value_counts()

## Multicollinearity


In [ ]:
import pandas as pd

corr = df[['Educ','Experience','Wage', 'Age']].corr()
print(corr)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of Numeric Variables")
plt.show()


In [75]:
# Create 2 datasets
df_exp = df2.drop("Age", axis = 1)
df_Age = df2.drop("Experience", axis = 1)


In [ ]:
df2.columns

## One-Hot-Encoding

In [82]:
df_exp_dummy = pd.get_dummies(df_exp, columns=['Race','Occup', 'Sector'], drop_first=True)
df_Age_dummy = pd.get_dummies(df_Age, columns=['Race','Occup', 'Sector'], drop_first=True)

In [ ]:
# Check it out!
df_Age_dummy.head()

## Making the model for df_exp_dummy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# 1. creating target and independent varibles
df_1 = df_exp_dummy.copy()
X= df_1.drop(columns = 'Wage')
y = df_1['Wage']

#2.  split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #seed number =It’s simply a seed number that tells Python to generate the same random results every time you run the code.

#3.   fit
model = LinearRegression() # Create a LinearRegression object and store it in a variable called model.”
model.fit(X_train, y_train)

# 4 predict & metrics
y_pred = model.predict(X_test)
print("MAE:", round(mean_absolute_error(y_test, y_pred), 3))
print("RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred)), 3))



In [ ]:
# predict an Unseen persin's wage
Unseen_Person = X_test.iloc[[1]]
X_test.iloc[[1]]

In [ ]:
# Predict it:
Prediction = model.predict(Unseen_Person)
print('Predicted wage is','$',round(Prediction[0],2))

##Making the model for df_Age_dummy

In [ ]:
# 1. creating target and independent varibles
df_2 = df_Age_dummy.copy()
X= df_2.drop(columns = 'Wage')
y = df_2['Wage']

#2.  split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #seed number =It’s simply a seed number that tells Python to generate the same random results every time you run the code.

#3.   fit
model2 = LinearRegression() # Create a LinearRegression object and store it in a variable called model.”
model2.fit(X_train, y_train)

4# predict & metrics
y_pred2 = model2.predict(X_test)
print("MAD:")
print("Model 2 (with Age) MAE:", round(mean_absolute_error(y_test, y_pred2), 3))
print("Model 1 MAE (with experience):", round(mean_absolute_error(y_test, y_pred), 3))
print("RMSE:")
print("Model 2 (with Age) RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred2)), 3))
print("Model 1 (with experience) RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred)), 3))



In [ ]:
!pip install langchain langchain-community ollama scikit-learn pydantic pandas

In [121]:
# make a clone of my model;

import copy

model_copy = copy.deepcopy(model)


In [125]:
# Saving teh model _copy
import pickle
# suppose your fitted model variable is called model2
pickle.dump(model_copy, open("model_copy.pkl", "wb"))


In [126]:
import pandas as pd

def predict_age(data_dict):
    df = pd.DataFrame([data_dict])
    return float(model_copy.predict(df)[0])


In [128]:
example = {
  "Educ":12,"South":0,"Gender":0,"Experience":30,"Union":0,"Marr":1,
  "Race_2":False,"Race_3":True,
  "Occup_2":False,"Occup_3":False,"Occup_4":False,"Occup_5":False,"Occup_6":False,
  "Sector_1":True,"Sector_2":False
}
print(predict_age(example))


14.353646726783758


In [131]:
!pip install -q transformers accelerate torch

In [135]:
# ===== 0) Lightweight, public LLM =====
!pip install -q transformers accelerate torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # public & small
tok = AutoTokenizer.from_pretrained(MODEL_ID)
llm = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
gen = pipeline("text-generation", model=llm, tokenizer=tok)

def llm_generate(prompt, max_new_tokens=192):
    # keep input short to avoid context overflow
    enc = tok(prompt, return_tensors="pt", truncation=True, max_length=512).to(llm.device)
    out = llm.generate(**enc, max_new_tokens=max_new_tokens, do_sample=False)
    return tok.decode(out[0], skip_special_tokens=True).strip()

# ===== 1) Your sklearn model wrapper =====
import pandas as pd

model = model_copy  # reuse your trained model in memory

FEATURE_ORDER = [
    "Educ","South","Gender","Experience","Union","Marr",
    "Race_2","Race_3","Occup_2","Occup_3","Occup_4","Occup_5","Occup_6",
    "Sector_1","Sector_2"
]

def predict_from_dict(d: dict) -> float:
    row = {}
    for k in FEATURE_ORDER:
        v = d[k]
        if isinstance(v, bool):  # convert booleans to 0/1
            v = int(v)
        row[k] = v
    X = pd.DataFrame([row], columns=FEATURE_ORDER)
    return float(model.predict(X)[0])

# ===== 2) Tiny, strict schema & robust JSON extraction =====
SCHEMA = (
    'Return ONLY compact JSON with one of these:\n'
    '{"action":"predict","data":{FEATURES}}\n'
    '{"action":"update_and_predict","updates":{FEATURES}}\n'
    'FEATURES keys must be exactly: '
    '["Educ","South","Gender","Experience","Union","Marr","Race_2","Race_3",'
    '"Occup_2","Occup_3","Occup_4","Occup_5","Occup_6","Sector_1","Sector_2"]. '
    'Booleans must be true/false, numbers are unquoted.'
)

import json, re

def extract_first_json(text: str) -> dict:
    # grab the first {...} block with a simple bracket counter
    start = text.find('{')
    if start == -1: raise ValueError(f"No JSON object found. Raw:\n{text}")
    depth = 0
    for i, ch in enumerate(text[start:], start):
        if ch == '{': depth += 1
        elif ch == '}':
            depth -= 1
            if depth == 0:
                candidate = text[start:i+1]
                break
    else:
        raise ValueError(f"Incomplete JSON. Raw:\n{text}")
    # normalize common glitches
    candidate = candidate.replace("True", "true").replace("False", "false")
    return json.loads(candidate)

# ===== 3) Stateful controller (predict / update_and_predict) =====
state = None

def controller(user_msg: str):
    global state
    prompt = (
        f"{SCHEMA}\n"
        f"User: {user_msg}\n"
        f"Assistant (JSON only):"
    )
    raw = llm_generate(prompt)
    obj = extract_first_json(raw)

    action = obj.get("action")
    if action == "predict":
        data = obj["data"]
        # ensure all required keys are present
        missing = [k for k in FEATURE_ORDER if k not in data]
        if missing:
            raise ValueError(f"Missing keys in data: {missing}")
        state = data.copy()
        pred = predict_from_dict(state)
        return {"prediction": pred, "state": state}

    elif action == "update_and_predict":
        if state is None:
            raise ValueError("No prior state. First do a 'predict' with full data.")
        updates = obj["updates"]
        for k, v in updates.items():
            if k not in FEATURE_ORDER:
                raise ValueError(f"Unknown feature: {k}")
            state[k] = v
        pred = predict_from_dict(state)
        return {"prediction": pred, "state": state}

    else:
        raise ValueError(f"Unknown action: {action}")


Device set to use cpu


In [ ]:
# 1) First prediction (full row)
msg1 = """predict for
Educ=12, South=0, Gender=0, Experience=23, Union=0, Marr=1,
Race_2=false, Race_3=true, Occup_2=false, Occup_3=false, Occup_4=false,
Occup_5=false, Occup_6=false, Sector_1=true, Sector_2=false
"""
out1 = controller(msg1)
print("Prediction:", round(out1["prediction"], 4))
print("State:", out1["state"])